## Import libraries & files

In [1]:
''' importing basic data analysis packages'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os,random, math, psutil, pickle 
import missingno as msno
from datetime import timedelta 
import plotly.express as px
import itertools

'''For Stat'''
import statsmodels.api as sm
from scipy.stats import chi2_contingency


''' For ML'''
from sklearn import metrics, svm
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
'''Reading in data'''
df = pd.read_pickle('tz_aware_merged_data_weather_imputed.pkl')

## picking out relevant features

In [3]:
df.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count', 'time_index',
       'day_of_week', 'hour_of_day', 'index', 'avg', 'std', 'outlier',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'timestamp_utc',
       'wind_direction', 'wind_speed', 'timezone', 'country_code', 'location',
       'dst', 'local_time'],
      dtype='object')

In [4]:
df_model = df[['building_id', 'timestamp', 'meter_reading', 'site_id', 'primary_use', 'square_feet', 'year_built',\
               'floor_count','day_of_week', 'hour_of_day','air_temperature', 'dew_temperature' ]]

In [5]:
# We decided to exclude buidling_id and timestamp 
df_model = df_model.drop(columns = ['building_id','timestamp'])

In [6]:
# get rid of negative meter_reading data 
df_model_nz = df_model.loc[df_model.meter_reading>0]

In [7]:
len(df_model_nz) / len(df_model)

0.9958662150229752

### Split data into training and test 

In [8]:
df_model_nz_x = df_model_nz.loc[:,~df_model_nz.columns.isin(['meter_reading'])]
df_model_nz_y = df_model_nz.loc[:,df_model_nz.columns.isin(['meter_reading'])]

In [9]:
# spliting the clean dataset into 70/30 for training/ test
X_train, X_test, y_train, y_test = train_test_split(df_model_nz_x, df_model_nz_y, test_size=0.3, random_state=1)

## Model 1

### Picking only key variables & log transform y variable

In [10]:
# Picking out x-variables we want to explore
X_train_a = X_train[['square_feet', 'floor_count', 'primary_use','site_id', 'day_of_week','hour_of_day']]
X_test_a = X_test[['square_feet', 'floor_count', 'primary_use','site_id', 'day_of_week', 'hour_of_day']]

# converting categorical data into dummy variables
X_train_a = pd.get_dummies (X_train_a, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])
X_test_a = pd.get_dummies (X_test_a, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])

# Dropping one categorical dummy variables
X_train_a= X_train_a.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])
X_test_a= X_test_a.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])

### Model 1, linear regression no transformation

In [11]:
model = sm.OLS(y_train, X_train_a).fit()
predictions = model.predict(X_train_a) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          meter_reading   R-squared (uncentered):                   0.647
Model:                            OLS   Adj. R-squared (uncentered):              0.647
Method:                 Least Squares   F-statistic:                          2.521e+05
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        0.00
Time:                        11:31:38   Log-Likelihood:                     -5.8774e+07
No. Observations:             8408713   AIC:                                  1.175e+08
Df Residuals:                 8408652   BIC:                                  1.175e+08
Df Model:                          61                                                  
Covariance Type:            nonrobust                                                  
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
square_feet                                   0.0015   9.52e-07   1538.730      0.000       0.001       0.001
floor_count                                  32.8810      0.037    884.371      0.000      32.808      32.954
primary_use_Education                       -47.2678      0.541    -87.318      0.000     -48.329     -46.207
primary_use_Entertainment/public assembly   -73.6493      0.575   -128.194      0.000     -74.775     -72.523
primary_use_Food sales and service          -24.3975      1.595    -15.297      0.000     -27.524     -21.271
primary_use_Healthcare                        9.1025      0.917      9.927      0.000       7.305      10.900
primary_use_Lodging/residential            -130.0460      0.576   -225.970      0.000    -131.174    -128.918
primary_use_Manufacturing/industrial        -42.7992      1.193    -35.869      0.000     -45.138     -40.461
primary_use_Office                          -73.8187      0.556   -132.662      0.000     -74.909     -72.728
primary_use_Other                           -81.4794      0.877    -92.856      0.000     -83.199     -79.760
primary_use_Parking                        -507.9516      0.936   -542.694      0.000    -509.786    -506.117
primary_use_Public services                 -79.1739      0.595   -133.161      0.000     -80.339     -78.009
primary_use_Religious worship                 2.8230      2.059      1.371      0.170      -1.213       6.859
primary_use_Retail                          -35.3831      1.181    -29.972      0.000     -37.697     -33.069
primary_use_Services                       -327.3538      1.255   -260.942      0.000    -329.813    -324.895
primary_use_Technology/science              -97.7999      1.571    -62.273      0.000    -100.878     -94.722
primary_use_Utility                          55.7732      1.805     30.897      0.000      52.235      59.311
site_id_0                                    96.3378      0.517    186.497      0.000      95.325      97.350
site_id_1                                  -102.7411      0.601   -170.922      0.000    -103.919    -101.563
site_id_2                                  -127.4762      0.455   -279.978      0.000    -128.369    -126.584
site_id_3                                   -89.1725      0.408   -218.688      0.000     -89.972     -88.373
site_id_4                                   -81.4304      0.508   -160.149      0.000     -82.427     -80.434
site_id_5                                   -68.1775      0.493   -138.391      0.000     -69.143     -67.212
site_id_6                                    -2.3680      0.650     -3.642      0.000      -3.642      -1.094
site_id_7     

In [22]:
# has 18% negative predictions.
sum(predictions<0)/len(predictions)

0.1769229131735142

### Model 1, with y variable log transformed

In [23]:
# log transform
y_train_a = y_train['meter_reading'].apply(np.log)

In [24]:
model = sm.OLS(y_train_a, X_train_a).fit()
predictions_log = model.predict(X_train_a) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          meter_reading   R-squared (uncentered):                   0.931
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                          1.848e+06
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        0.00
Time:                        11:38:55   Log-Likelihood:                     -1.3281e+07
No. Observations:             8408713   AIC:                                  2.656e+07
Df Residuals:                 8408652   BIC:                                  2.656e+07
Df Model:                          61                                                  
Covariance Type:            nonrobust                                                  
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
square_feet                                6.735e-06   4.26e-09   1581.432      0.000    6.73e-06    6.74e-06
floor_count                                   0.1191      0.000    716.724      0.000       0.119       0.119
primary_use_Education                         2.5320      0.002   1046.188      0.000       2.527       2.537
primary_use_Entertainment/public assembly     1.9711      0.003    767.356      0.000       1.966       1.976
primary_use_Food sales and service            2.7402      0.007    384.266      0.000       2.726       2.754
primary_use_Healthcare                        2.6705      0.004    651.416      0.000       2.663       2.679
primary_use_Lodging/residential               2.1229      0.003    825.046      0.000       2.118       2.128
primary_use_Manufacturing/industrial          2.8213      0.005    528.858      0.000       2.811       2.832
primary_use_Office                            2.2882      0.002    919.757      0.000       2.283       2.293
primary_use_Other                             1.5431      0.004    393.319      0.000       1.535       1.551
primary_use_Parking                           0.4098      0.004     97.934      0.000       0.402       0.418
primary_use_Public services                   2.4026      0.003    903.797      0.000       2.397       2.408
primary_use_Religious worship                 0.3777      0.009     41.026      0.000       0.360       0.396
primary_use_Retail                            1.8567      0.005    351.784      0.000       1.846       1.867
primary_use_Services                          1.3832      0.006    246.606      0.000       1.372       1.394
primary_use_Technology/science                2.9806      0.007    424.485      0.000       2.967       2.994
primary_use_Utility                           3.2563      0.008    403.477      0.000       3.241       3.272
site_id_0                                     1.0210      0.002    442.068      0.000       1.016       1.025
site_id_1                                    -0.0091      0.003     -3.399      0.001      -0.014      -0.004
site_id_2                                    -0.0915      0.002    -44.969      0.000      -0.096      -0.088
site_id_3                                    -0.2995      0.002   -164.286      0.000      -0.303      -0.296
site_id_4                                    -0.4577      0.002   -201.341      0.000      -0.462      -0.453
site_id_5                                    -1.3399      0.002   -608.331      0.000      -1.344      -1.336
site_id_6                                     0.4728      0.003    162.622      0.000       0.467       0.478
site_id_7     

In [29]:
y_pred_log = model.predict(X_test_a)
y_pred_log_trans = y_pred_log .apply(np.exp)

In [47]:
# Printing RMSLE 
y_true = y_test['meter_reading'].values.tolist()
y_p = y_pred_log_trans.ravel().tolist()

np.sqrt(metrics.mean_squared_log_error(y_true, y_p))

1.0132668198456

### Model 2, including weather data

In [48]:
# Picking out x-variables we want to explore
X_train_b = X_train[['square_feet', 'floor_count', 'primary_use','site_id', 'day_of_week','hour_of_day','air_temperature']]
X_test_b = X_test[['square_feet', 'floor_count', 'primary_use','site_id', 'day_of_week', 'hour_of_day','air_temperature']]

# converting categorical data into dummy variables
X_train_b = pd.get_dummies (X_train_b, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])
X_test_b = pd.get_dummies (X_test_b, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])

# Dropping one categorical dummy variables
X_train_b= X_train_b.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])
X_test_b= X_test_b.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])

In [49]:
# log transform
y_train_b = y_train['meter_reading'].apply(np.log)

In [50]:
model = sm.OLS(y_train_b, X_train_b).fit()
predictions_log = model.predict(X_train_b) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          meter_reading   R-squared (uncentered):                   0.931
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                          1.820e+06
Date:                Mon, 09 Dec 2019   Prob (F-statistic):                        0.00
Time:                        11:58:30   Log-Likelihood:                     -1.3278e+07
No. Observations:             8408713   AIC:                                  2.656e+07
Df Residuals:                 8408651   BIC:                                  2.656e+07
Df Model:                          62                                                  
Covariance Type:            nonrobust                                                  
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
square_feet                                6.735e-06   4.26e-09   1582.229      0.000    6.73e-06    6.74e-06
floor_count                                   0.1190      0.000    715.830      0.000       0.119       0.119
air_temperature                               0.0039   4.67e-05     84.288      0.000       0.004       0.004
primary_use_Education                         2.5048      0.002   1026.243      0.000       2.500       2.510
primary_use_Entertainment/public assembly     1.9438      0.003    751.160      0.000       1.939       1.949
primary_use_Food sales and service            2.7129      0.007    380.213      0.000       2.699       2.727
primary_use_Healthcare                        2.6435      0.004    643.131      0.000       2.635       2.652
primary_use_Lodging/residential               2.0951      0.003    807.985      0.000       2.090       2.100
primary_use_Manufacturing/industrial          2.7935      0.005    522.877      0.000       2.783       2.804
primary_use_Office                            2.2609      0.003    901.563      0.000       2.256       2.266
primary_use_Other                             1.5157      0.004    385.200      0.000       1.508       1.523
primary_use_Parking                           0.3824      0.004     91.153      0.000       0.374       0.391
primary_use_Public services                   2.3757      0.003    887.743      0.000       2.370       2.381
primary_use_Religious worship                 0.3488      0.009     37.883      0.000       0.331       0.367
primary_use_Retail                            1.8290      0.005    345.993      0.000       1.819       1.839
primary_use_Services                          1.3558      0.006    241.428      0.000       1.345       1.367
primary_use_Technology/science                2.9550      0.007    420.622      0.000       2.941       2.969
primary_use_Utility                           3.2272      0.008    399.668      0.000       3.211       3.243
site_id_0                                     0.9710      0.002    407.395      0.000       0.966       0.976
site_id_1                                    -0.0154      0.003     -5.714      0.000      -0.021      -0.010
site_id_2                                    -0.1505      0.002    -69.953      0.000      -0.155      -0.146
site_id_3                                    -0.3219      0.002   -174.795      0.000      -0.326      -0.318
site_id_4                                    -0.4782      0.002   -209.261      0.000      -0.483      -0.474
site_id_5                                    -1.3437      0.002   -610.187      0.000      -1.348      -1.339
site_id_6     

In [51]:
y_pred_b_log = model.predict(X_test_b)
y_pred_b = y_pred_b_log .apply(np.exp)
# Printing RMSLE 
y_true = y_test['meter_reading'].values.tolist()
y_p = y_pred_b.ravel().tolist()

np.sqrt(metrics.mean_squared_log_error(y_true, y_p))

1.012756303879108

### Model 3:  SKLearn linear ML

In [53]:
# Picking out x-variables we want to explore
X_train_c = X_train.copy()
X_test_c = X_test.copy()

# converting categorical data into dummy variables
X_train_c = pd.get_dummies (X_train_c, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])
X_test_c = pd.get_dummies (X_test_c, columns = ['primary_use','site_id','day_of_week', 'hour_of_day'])

# Dropping one categorical dummy variables
X_train_c= X_train_c.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])
X_test_c= X_test_c.drop(columns = ['primary_use_Warehouse/storage', 'site_id_15', 'day_of_week_6', 'hour_of_day_23' ])

In [54]:
y_train_c = y_train['meter_reading'].apply(np.log)

In [55]:
# Training model using linear regression
regressor = LinearRegression()  
regressor.fit(X_train_c, y_train_c)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [56]:
# predicting y 
y_pred = regressor.predict(X_test_c)


In [61]:
y_pred_series  = pd.Series(y_pred)

In [77]:
y_pred_trans = y_pred_series .apply(np.exp)
y_pred_array = y_pred_trans.array

# Printing RMSLE 
y_true = y_test['meter_reading'].values.tolist()
y_p = y_pred_trans.ravel().tolist()

In [64]:
np.sqrt(metrics.mean_squared_log_error(y_true, y_p))

0.9824119924764476